<a href="https://colab.research.google.com/github/sjeom6/Colab_Repo/blob/main/langchain_240728_IMG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LangChain OpenAI 테스트

In [7]:
!pip install openai
!pip install Pillow
!pip install matplotlib
!pip install requests

In [8]:
import os
import openai

os.environ["OPENAI_API_KEY"] = ""

client = openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [15]:
import os
import openai
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

# OpenAI API 키 설정
openai.api_key = "YOUR_API_KEY"

client = openai.OpenAI()
image_url = "https://noonnucc-production.sfo2.cdn.digitaloceanspaces.com/202306/1685628411919633.jpeg"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text",
                 "text": "한국 자동차 이미지 인데 이 그림에 번호판 글 추출해줘"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url" : image_url
                    }
                }
            ]
        }
    ],
    max_tokens=1000
)

# 이미지 다운로드
# download_img = requests.get(image_url)
# img = Image.open(BytesIO(download_img.content))

# 이미지 출력
# plt.imshow(img)
# plt.axis('off') # 축 정보 숨기기
# plt.show()

# 응답 출력
print(response.choices[0].message.content)

번호판의 글자는 다음과 같습니다:

**180포 3682**
